In [ ]:
'''
More selectively pick the sources used as a training set, as done in the Alhassan and Aniyan papers.

When selecting only include classifications I am sure about / that I would want the NN to recognise.
I.e. if I am uncertain don't include... as if did and NN classified it wrong would look bad on accuracy...

Classifications: FR1, FR2, BENT, COMP

Data samples: FR1CAT, FR2CAT, CoNFIG, Proctor 2011

1. Use only 'confirmed' sources from CoNFIG and Proctor.
2. Remove sources which are too large or uncentred in cutout.
3. Remove images (proctor) which contain multiple sources.
4. Check images don't occur in multiple samples (i.e. same image isn't in both FR1CAT and Proctor WAT/NAT)
'''

import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.io import fits
import pandas as pd
import matplotlib.colors as colors

In [ ]:
def display_dir_images(dir_path, cols=5, n=None, log=False, obj=None, save=None):
    # displays FITS images in the specified directory
    files = [f for f in os.listdir(dir_path) if f.endswith('.fits')]
    files = np.sort(files)
    if obj != None: files = np.array(files)[obj]
    if n == None: n = len(files)

    fig = plt.figure(figsize=(12.5,(n+cols-1)//cols*2.5))
    for i in range(n):
        ax = fig.add_subplot((n+cols-1)//cols, cols, i+1)
        filename = os.fsdecode(files[i])
        filepath = os.path.join(dir_path, filename)
        data = fits.getdata(filepath, ext=0)
        if log:
            data[np.isnan(data)] = 0
            data = (data-np.min(data))/np.ptp(data)+0.01 # normalises image for log plotting
            ax.imshow(data, cmap='inferno', norm=colors.LogNorm())
        else:
            ax.imshow(data, cmap='inferno')
        ax.text(0.9, 0.9, i, ha='center', va='center', transform=ax.transAxes, color='w')
        plt.axis('off')
    fig.tight_layout()
    if save != None: plt.savefig(save)
    plt.show()
    plt.close()
    
def fits_to_array(dir_path, obj=None, del_nan=True):
    '''
    Loads FITS files from a directory into an array
    Doesn't include images with nan values, outputs nan image indices
    '''
    files = [f for f in os.listdir(dir_path) if f.endswith('.fits')]
    files = np.sort(files)
    if obj != None: files = np.array(files)[obj]
    n = len(files)

    xpix, ypix = fits.getdata(os.path.join(dir_path,files[0]), ext=0).shape
    data = np.empty(shape=(n,xpix,ypix),dtype=np.float64)
    nan_indices = []
    for i in range(n):
        filepath = os.path.join(dir_path, files[i])
        data[i,:,:] = fits.getdata(filepath, ext=0)
        if np.isnan(data[i,:,:]).any():
            nan_indices.append(i)
            data[i,:,:][np.isnan(data[i,:,:])] = 0
    if del_nan: data = np.delete(data, nan_indices, axis=0)
    return np.array([data, nan_indices])

In [ ]:
# CoNFIG object type indices

filepath = '../../Data/CoNFIG.tsv'
data = pd.read_csv(filepath, sep='|', header=None, skiprows=3)
data = np.array(data)

# Array is ordered by sample (1,2,3,4), but saved data is ordered by ra
# So need to change order of array to be ordered by ra to match saved data
# (Are other possible approaches...)
data = data[np.argsort(data[:,4])]

fr1 = [i for i in range(len(data)) if data[i,8] == 'I ' and data[i,10] == 'c']
fr2 = [i for i in range(len(data)) if data[i,8] == 'II' and data[i,10] == 'c']
comp = [i for i in range(len(data)) if data[i,8] in ('C ','C*','S*')]

In [ ]:
## FR1 = FR1CAT (219 -> 197) + CoNFIG FR1s (all=71, confirmed=50 -> 23)

# FR1CAT
dir_path = r'\\its-rds.bham.ac.uk\rdsprojects\s\stevenir-radio-astronomy-neural-nets\data\FRICAT\FIRST'
dir_path = r'D:\FRICAT\FIRST'
#display_dir_images(dir_path, log=True, n=None)

# Manually picked out bad images by visual inspection
bent=[66, 93, 104, 141, 145, 153, 161, 199]
uncert=[45, 105]
multiple=[108, 109, 167, 168]
not_great=[2, 15, 82, 91, 116, 139, 144, 171] # very dim / other nearby bright spots

fr1cat_data = fits_to_array(dir_path) # 219, no nan indices
fr1cat_data = np.delete(fr1cat_data[0], bent+uncert+multiple+not_great, axis=0) # 197


# CoNFIG FR1s (only included 'confirmed' FR1s)
dir_path = r'\\its-rds.bham.ac.uk\rdsprojects\s\stevenir-radio-astronomy-neural-nets\data\CoNFIG\FIRST'
dir_path = r'D:\CoNFIG\FIRST'
#display_dir_images(dir_path, log=True, n=None, obj=fr1)

# Manually picked out bad images by visual inspection
bent=[10, 18, 21, 25, 26, 30]
uncert=[3, 7, 17, 22, 29, 31, 33, 34, 35, 39, 40, 41, 49] # unresolved so uncertain
fr2_=[1, 2, 4, 13, 19, 24] # some of these looked more like fr2 to me...
multiple=[]
not_great=[8, 43] # very dim / other nearby bright spots

config_fr1_data = fits_to_array(dir_path, obj=fr1) # 50, no nan indices
config_fr1_data = np.delete(config_fr1_data[0], bent+uncert+fr2_+multiple+not_great, axis=0) # 23

images = np.concatenate((fr1cat_data, config_fr1_data))

print(fr1cat_data.shape, config_fr1_data.shape, images.shape)
#np.save('good_fr1.npy', images)

In [ ]:
## FR2 = FR2CAT (123 -> 111 (+10 large)) + CoNFIG FR2s (all=466, confirmed=390 -> 294 (+6 large))

# FR1CAT
dir_path = r'\\its-rds.bham.ac.uk\rdsprojects\s\stevenir-radio-astronomy-neural-nets\data\FRIICAT\FIRST'
#dir_path = r'D:\FRIICAT\FIRST'
#display_dir_images(dir_path, log=True, n=None)

# Manually picked out bad images by visual inspection
bent=[]
uncert=[56]
multiple=[]
not_great=[75] # very dim / other nearby bright spots

# Lots of large FR2, but don't really want to discard, should just not crop these images (only resize)
large=[3, 11, 41, 46, 55, 60, 70, 94, 96, 105]

fr2cat_data = fits_to_array(dir_path) # 123, no nan indices
large_fr2cat_data = fr2cat_data[0][large] # 10
fr2cat_data = np.delete(fr2cat_data[0], bent+uncert+multiple+not_great+large, axis=0) # 111


# CoNFIG FR2s (only included 'confirmed' FR2s)
dir_path = r'\\its-rds.bham.ac.uk\rdsprojects\s\stevenir-radio-astronomy-neural-nets\data\CoNFIG\FIRST'
#dir_path = r'D:\CoNFIG\FIRST'
#display_dir_images(dir_path, log=True, n=None, obj=fr2)

# Manually picked out bad images by visual inspection
bent=[]
uncert=[2, 4, 7, 11, 16, 17, 18, 22, 26, 32, 35, 36, 39, 41, 50, 54, 59, 61, 62, 63, 72, 92, 93, 95,
        97, 99, 102, 128, 140, 148, 154, 155, 159, 162, 164, 165, 169, 173, 175, 177, 179, 184,
        185, 194, 196, 202, 204, 221, 223, 224, 231, 232, 234, 237, 250, 252, 262, 265, 271, 273,
        276, 279, 282, 286, 291, 292, 294, 295, 298, 299, 300, 302, 304, 306, 308, 312, 314, 320,
        323, 324, 326, 330, 334, 335, 342, 362, 368, 372, 373] # unresolved or weird so uncertain
multiple=[123]
not_great=[] # very dim / other nearby bright spots

large=[14, 94, 142, 143, 146, 254]

config_fr2_data = fits_to_array(dir_path, obj=fr2) # 390, no nan indices
large_config_fr2_data = config_fr2_data[0][large] # 6
config_fr2_data = np.delete(config_fr2_data[0], bent+uncert+multiple+not_great+large, axis=0) # 294

images = np.concatenate((fr2cat_data, config_fr2_data))

print(fr2cat_data.shape, config_fr2_data.shape, images.shape)
#np.save('good_fr2.npy', images)

In [ ]:
## BENT = WATs and NATs from Proctor 2011 (412 -> 226)

dir_path = r'\\its-rds.bham.ac.uk\rdsprojects\s\stevenir-radio-astronomy-neural-nets\data\proctor-2011\1-WAT-NAT'
dir_path = r'D:\proctor-2011\1-WAT-NAT'
#display_dir_images(dir_path, log=True, n=None)

# Manually picked out bad images by visual inspection
not_bent=[1,9,13,15,29,30,34,46,51,93,94,95,98,100,104,107,109,122,127,129,132,134,141,
          166,175,181,187,189,198,199,200,210,229,233,237,243,244,249,256,258,288,292,
          308,316,346,354,358,363,367,372,373,407]
uncert=[10,12,19,23,24,43,44,53,54,69,79,81,84,86,90,99,108,111,112,113,114,121,125,136,
        156,157,158,159,167,174,176,182,183,221,225,230,232,234,238,257,264,274,275,278,
        280,281,284,285,294,298,303,304,305,306,311,312,313,318,322,324,325,332,353,356,360,361,362,393,400]
multiple=[6,7,27,28,38,39,64,65,66,76,77,101,194,195,196,197,267,268,319,328,402,408]
duplicate=[289,376,387,389]
not_great=[20,32,33,35,40,52,75,82,92,102,110,118,135,138,139,140,143,149,153,179,188,
           191,193,203,205,209,236,248,260,266,273,277,283,302,336,357,383,390,391]


bent_data = fits_to_array(dir_path, del_nan=False) # 412, 2 with nan values, but both useable (so set nan=0)
bent_data = np.delete(bent_data[0], not_bent+uncert+multiple+duplicate+not_great, axis=0) # 226

print(bent_data.shape)
#np.save('good_bent.npy', bent_data)

In [ ]:
## COMP = C,C*,S* from CoNFIG (284 -> 243)
dir_path = r'D:\CoNFIG\FIRST'
#display_dir_images(dir_path, log=True, n=None, obj=comp)

not_comp=[44,36,61,62,71,80,82,84,88,11,113,116,125,127,128,131,132,138,139,141,143,
          146,153,158,163,171,183,204,207,208,216,225,230,236,238,240,241,257,264,265,283]

comp_data = fits_to_array(dir_path, obj=comp) # 284, no nan indices
comp_data = np.delete(comp_data[0], not_comp, axis=0) # 243

print(comp_data.shape)
#np.save('good_comp.npy', comp_data)

In [5]:
not_comp=[44,36,61,62,71,80,82,84,88,11,113,116,125,127,128,131,132,138,139,141,143,
          146,153,158,163,171,183,204,207,208,216,225,230,236,238,240,241,257,264,265,283]
len(not_comp)

41

In [ ]:
## Plot ConFIG images with labels

# Variables
dir_path = r'\\its-rds.bham.ac.uk\rdsprojects\s\stevenir-radio-astronomy-neural-nets\data\CoNFIG\FIRST'
dir_path = r'D:\CoNFIG\FIRST'

cols = 5
n = 10
log = True

# Image info
filepath = '../../Data/CoNFIG.tsv'
data = pd.read_csv(filepath, sep='|', header=None, skiprows=3)
data = np.array(data)
data = data[np.argsort(data[:,4])] # sort by ra to match directory

# Display images
files = [f for f in os.listdir(dir_path) if f.endswith('.fits')]
files = np.sort(files) 
if n == None: n = len(files)

fig = plt.figure(figsize=(12.5,(n+cols-1)//cols*2.5))
for i in range(n):
    ax = fig.add_subplot((n+cols-1)//cols, cols, i+1)
    filename = os.fsdecode(files[i])
    filepath = os.path.join(dir_path, filename)
    im = fits.getdata(filepath, ext=0)
    if log:
        im = (im-np.min(im))/np.ptp(im)+0.01 # normalises image for log plotting
        ax.imshow(im, cmap='inferno', norm=colors.LogNorm())
    else:
        ax.imshow(im, cmap='inferno')
    ax.text(0.9, 0.9, i, ha='center', va='center', transform=ax.transAxes, color='w')
    ax.set_title(data[i,8])
    plt.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
## Display images saved in .npy files

def display_npy_images(npy_path, cols=5, n=None, log=False, save=None):
    images = np.load(npy_path, allow_pickle=True)
    print(images.shape)
    if n == None: n = len(images)
    fig = plt.figure(figsize=(12.5,(n+cols-1)//cols*2.5))
    for i in range(n):
        ax = fig.add_subplot((n+cols-1)//cols, cols, i+1)
        data = images[i]
        if log:
            data[np.isnan(data)] = 0
            data = (data-np.min(data))/np.ptp(data)+0.01 # normalises image for log plotting
            ax.imshow(data, cmap='inferno', norm=colors.LogNorm())
        else:
            ax.imshow(data, cmap='inferno')
        ax.text(0.9, 0.9, i, ha='center', va='center', transform=ax.transAxes, color='w')
        plt.axis('off')
    fig.tight_layout()
    if save != None: plt.savefig(save)
    plt.show()
    plt.close()
    
npy_path = '../../Data/Good/good_fr2.npy'
#display_npy_images(npy_path, log=True)